<hr style="border: solid 1px red; margin-bottom: 2% ">

### ARCHER SCIENTIFIC PYTHON COURSE:

# C/Fortran Interface

<hr style="border: solid 1px red; margin-bottom: -1%; ">

## Website:  http://www.archer.ac.uk 

## Helpdesk: support@archer.ac.uk

<br>

<img src="../images/epsrclogo.png" style="width: 40%;">
<br>
<img src="../images/nerclogo.png" style="width: 40%;">

<br>
<img src="../images/craylogo.png" style="width: 40%;">

<br>
<img src="../images/epcclogo.png" style="width: 40%;">

<br>
<img src="../images/ediunilogo.png" style="width: 40%" > 

<br>
<br>

<img src="../images/reusematerial.png" style="width: 90">
<br>
<br>

<br>
<hr class="top">

# C/Fortran Interface

<hr class="bot">
<br>

## Presenter: Adrian Jackson

#### Contributing authors:
#### Adrian Jackson, Neelofer Banglawala, Arno Proeme, Kevin Stratford, Andy Turner

<br>
<br>
<br>



<br>
<hr class="top">

## Why couple python with another language?

<hr class="bot">
<br>

* Combine performance of compiled codes with flexibility of Python

  * Accelerate code where python is slow
  * Incorporate Python analysis and visualisation into existing codebase
  * Provide flexible way to extract results from code using Python 


* Reuse code that you already have

  * Gradually introduce new functionality using Python


* There are a number of ways to do this (quite a large number)

  * One way or another might suit your needs
  * All doing essentially the same thing

<br>
<br>

## Importing modules from shared libraries


* Basic approach is to compile *a shared library*

  * Compiles native langauge source 
  * Describe the interface to python


* Requires

  * Appropriate compiler (e.g., `gfortran`, `cc`, `...`)
  * Usually `-fPIC` Position Independent Code


* Will produce

  * A shared library (`.so` in Unix; DLLs in Windows) to be loaded at run time
  * Some approaches `import` module as usual


* Some care may be required with compound/opaque data types
  * A clear picture of the number of types of any arguments
  
<br>
<br>


## Python and C via `ctypes`


* `ctypes` is a python standard library module

    * Provides utilities to describe C-compatible data types
    * Manipulation of shared libraries/DLLs
    * Uses only python (no additional files/mixed-language intermediate code)

```python
    import ctypes
```


* Must load the library (`.so`) file explicitly

```python
    lib = ctypes.cdll.LoadLibrary("./my_library.so")
```


* Must specify the prototype for the C function, e.g.,

```python
    # Result type
    lib.my_c_function.restype = ctypes.c_int
    # Argument list
    lib.my_c_function.argtypes = [ctypes.c_double]
```

* Potentially error-prone

<br>
<br>


## ctypes example: C side

Consider the simple function:
```c
int my_c_function(double val) {

    return (int) (val + 1.0);
}
```

We need to compile a shared library:

<br>

In [ ]:

# Issue the appropriate compilation commands
# and check the library exists...

! gcc -c -fPIC my_library.c
! gcc -shared -o my_library.so my_library.o
! ls -l my_library.so



<br>
<br>


## ctypes example: python side

Once we have the shared library:



In [ ]:

import ctypes

lib = ctypes.cdll.LoadLibrary("./my_library.so")

# Define return type ...
lib.my_c_function.restype = ctypes.c_int

# ...and the argument list
lib.my_c_function.argtypes = [ctypes.c_double]

x = float(23)
result = lib.my_c_function(x)
print(result, type(result))



<br>
<br>

## Arrays


Consider the following example in C with two pointer arguments:
    
```c
#include <math.h>

void array_sqrt(int n, double * a_in, double * a_out) {

  int i;

  for (i = 0; i < n; i++) {
    a_out[i] = sqrt(a_in[i]);
  }

  return;
}
```



In [ ]:

# Generate the shared library as before
! gcc -c -fPIC c_sqrt.c
! gcc -shared -o c_sqrt.so c_sqrt.o
! ls -l c_sqrt.so


<br>
<br>

## Using ctypes



In [ ]:
# We need a pointer type p_double
# Use ctypes utility function POINTER()

import ctypes

c_int = ctypes.c_int
c_double = ctypes.c_double
p_double = ctypes.POINTER(c_double)

liba = ctypes.cdll.LoadLibrary("./c_sqrt.so")

liba.array_sqrt.restype = None
liba.array_sqrt.argtypes = [c_int, p_double, p_double]



<br>
<br>

How do we extract something that looks like a C pointer from a numpy array?


In [ ]:

import numpy as np

# Define some test data as numpy arrays

a_in = np.array([16.0, 25.0, 36.0, 49.0])
a_out = np.zeros(4, np.double)

# Then we need to pass pointers to actual data
pin  =  a_in.ctypes.data_as(p_double)
pout = a_out.ctypes.data_as(p_double)

liba.array_sqrt(4, pin, pout)
print(a_out)


<br>
<br>


## Using `numpy.ctypeslib`


<br>

Couldn't we just pass the numpy `ndarray` as it is? 

In [ ]:

from numpy.ctypeslib import ndpointer

libb = ctypes.cdll.LoadLibrary("./c_sqrt.so")

# Describe a_in, a_out
ndarg = ndpointer(ctypes.c_double, flags = "C_CONTIGUOUS")

libb.array_sqrt.restype = None
libb.array_sqrt.argtypes = [c_int, ndarg, ndarg]

# A new output array
a_out = np.zeros(4, np.double, order = 'C')

libb.array_sqrt(4, a_in, a_out)
print(a_out)



<br>
<br>

## Fortran via ctypes

It is possible to call external Fortran procedures using a ctypes description.

Consider an analogue of the square root routine
```fortran
subroutine array_sqrt(n, ain, aout)

  implicit none
  integer, intent(in)  :: n
  real,    intent(in)  :: ain(n)   ! Note default real
  real,    intent(out) :: aout(n)  ! Ditto
  
  integer :: i
  
  do i = 1, n
    aout(n) = ain(n)**0.5
  end do
  
  return
end subroutine array_sqrt

```

<br>
<br>

In [ ]:

# Compile a shared library
! gfortran -c -fPIC farray_sqrt.f90
! gfortran -shared -fPIC -o farray_sqrt.so farray_sqrt.o
! ls -l farray_sqrt.so


In [ ]:

# Python

import ctypes

libf = ctypes.cdll.LoadLibrary("farray_sqrt.so")

# p_int is required for call by reference
# p_float is required

c_int = ctypes.c_int
c_float = ctypes.c_float

p_int = ctypes.POINTER(c_int)
p_float = ctypes.POINTER(c_float)

# Note "name mangling" additional underscore

libf.array_sqrt_.restype = None
libf.array_sqrt_.argtypes = [p_int, p_float, p_float]


In [ ]:

import numpy as np

# generate some numpy test data
ain  = np.array([4.0, 9.0, 16.0, 25.0, 2.0], dtype = np.float32,
                order = 'F')
aout = np.zeros(ain.size, dtype = np.float32, order = 'F')

# Identify the ctypes pointer descriptions
pin  = ain.ctypes.data_as(p_float)
pout = aout.ctypes.data_as(p_float)

n = ctypes.c_int(ain.size)
libf.array_sqrt_(ctypes.byref(n), pin, pout)

print(aout)



<br>
<br>

### Note on development

Once loaded, there is no easy way to "re-load" a shared library
in a running process.

For the notebook, this means if the library is re-compiled, the only
way to re-load the new library in a reliable fashion is to restart
the kernel. This can be slightly tedious, so dealing with shared
libraries is perhaps better handled via command line and a script
which automatically means you have a new process each time.

<br>
<br>

## Using CFFI

* C foreign function interface is standard library module

    * Can load shared library directly ("ABI")
    * Can deal with code ("API")


* Application binary interface

    * Load shared library
    * Describe function prototypes


* Application programming interface

    * Provide prototype and source code
    * Compile from python
    * `import` resultant module
    
    
<br>
<br>


## cffi ABI (Application Binary Interface)

This is simliar to the ctypes approach:



In [ ]:
# Use C c_sqrt.so compiled above

import cffi

# The prototype must be provided
ffi = cffi.FFI()
ffi.cdef("void array_sqrt(int n, double * a_in, double * a_out);")

libc3 = ffi.dlopen("./c_sqrt.so")

# Provide test data
ain  = np.array([1.0, 2.0, 3.0, 4.0])
aout = np.zeros(ain.size)

pin  = ffi.cast("double *", ain.ctypes.data)
pout = ffi.cast("double *", aout.ctypes.data)

libc3.array_sqrt(4, pin, pout)

print(aout)



<br>
<br>

## cffi API (Application Programming Inferface)

This avoids explicit maniplulation of shared libraries:

In [ ]:

import cffi

# Provide prototype and code!
ffi = cffi.FFI()
ffi.cdef("void array_sqrt(int n, double * a_in, double * a_out);")

with open("./c_sqrt.c", "r") as srcfile:
    src = srcfile.read()

# Define a module name and compile
# Note: do not supply same name as original source file
#       or it will be overwritten

ffi.set_source("c_sqrt_ffi", src)
ffi.compile(verbose = True)


In [ ]:

# Import the library object of the new module

from c_sqrt_ffi import lib

ain  = np.array([2.0, 81.0])
aout = np.zeros(ain.size)

pin  = ffi.cast("double *", ain.ctypes.data)
pout = ffi.cast("double *", aout.ctypes.data)

lib.array_sqrt(ain.size, pin, pout)

print(aout)


<br>
<br>


## Using numba

The cffi approach might ease some portability issues, but looks much like
ctypes behind the scenes.

* The numba project uses a slightly different approach

    * Just-in-time compilation
    * In-built version of LLVM so no calls to external compiler/loading libraries

```python
from numba import jit
```

* Use `@jit` decorator to indicate functions for compilation

    * @jit (nopython = True)


In [ ]:

import numpy as np
from numba import jit

@jit (nopython = True)
def array_sqrt(n, ain, aout):

    for i in range(n):
        aout[i] = np.sqrt(ain[i])


a_in  = np.array([4.0, 9.0, 16.0, 25.0], np.double)
a_out = np.zeros(4, np.double)

array_sqrt(a_in.size, a_in, a_out)

print(a_out)


<br>

## Alternatives

<br>

* <p style="font-size: 100%"><b>Native Python interface</b></p>

  * <p style="font-size: 100%">Fully-flexible and portable</p>
  * <p style="font-size: 100%">Complex and verbose</p>
  * <p style="font-size: 100%">Option if you are interfacing a large amount of code and/or have a large software development project</p>


* <p style="font-size: 100%"><b>Cython</b> : converts Python-like code into a C library which can call other C libraries</p>

  * <p style="font-size: 100%">Standard C-like Python (or Python-like C)</p>
  
  
* <p style="font-size: 100%"> <b>SWIG</b> (or <b>S</b>implified <b>W</b>rapper and <b>I</b>nterface <b>G</b>enerator) : reads header files and generates a library Python can load </p>

  * <p style="font-size: 100%">Very generic and feature-rich</p>
  * <p style="font-size: 100%">Supports multiple languages other than Python (e.g. Perl, Ruby)</p>
  

  
  
    





* **Boost.python** : helps write C++ libraries that Python can load and use easily
  

  
    




<br>

## Summary

* Calling C/Fortran allows code re-use


* Fortran/C can give better performance than Python

* Modern Fortran users should consider `f90wrap`

<br>
<br>

https://github.com/jameskermode/f90wrap

<br>
<br>
<br>

<br>

## Exercise: `fibonacci.f90`

A C version is also available.
<br>

Using a method or methods of your choice, try to call
the following code from python.

`fibonacci()` computes the first `n` Fibonacci numbers: 0, 1, 1, 2, 3, 5, 8, 13,...
and stores the results in the array provided.

```fortran
subroutine fibonacci(n, a_out)
  implicit none
  integer, intent(in) :: n
  real*8, dimension(n) :: a_out

  integer :: i
  
  do i = 1, n
    if (i.eq.1) then
      a_out(i) = 0.0
    else if (i.eq.2) then
      a_out(i) = 1.0
    else
      a_out(i) = a_out(i-1) + a_out(i-2)
    end if
  end do
end subroutine fibonacci
```

You can try this on your laptop, or on ARCHER.

<br>

## Archer

<br>

Consult the sheet with your guest account details to login.

```
$ cd /work/y14/y14/guestNNN
$ git clone https://github.com/adrianjhpc/archer-python.git
```
and

```
$ cd archer-python/lectures/dev-compile
```
You will need an editor to edit files.



From the command line
```bash
bash> module swap PrgEnv-cray PrgEnv-gnu
bash> module swap gcc gcc/4.9.3
bash> module load anaconda
```

Use the commands from the cells above on the command line.

Create a script or use the ipython shell to run on the front end
<br>
<br>

<br>

## C Exercise

<br>

An equivalent C function is available to compute a Fibonacci series: `fibonacci.c`

Try using `ctypes` to call this function either using your laptop, or Archer.

If you are using your laptop, your will need a C compiler; on Archer, use `cc`.

<hr class="top">
<hr class="bot">
